# Pima-Diabetes-Prediction

In [108]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold 
from sklearn.model_selection import RepeatedKFold
from sklearn.cross_validation import train_test_split
from  sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.preprocessing import binarize
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from __future__ import print_function
import tensorflow as tf
from six.moves import range
import sys

from IPython.display import display, Image

# Config the matlotlib backend as plotting inline in IPython
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import seaborn as sns
from keras.models import Sequential

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix,classification_report

from tensorflow.contrib import keras
from keras import models, layers, losses, optimizers, metrics
import os

In [57]:
col_names = ['pregnant', 'glucose', 'bp', 'skin', 'insulin', 'bmi', 'pedigree', 'age', 'label']
data = pd.read_csv('pima-indians-diabetes.data', names=col_names)


In [58]:
data.count()

pregnant    768
glucose     768
bp          768
skin        768
insulin     768
bmi         768
pedigree    768
age         768
label       768
dtype: int64

In [59]:
data.head()

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age,label
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [60]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
pregnant    768 non-null int64
glucose     768 non-null int64
bp          768 non-null int64
skin        768 non-null int64
insulin     768 non-null int64
bmi         768 non-null float64
pedigree    768 non-null float64
age         768 non-null int64
label       768 non-null int64
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [61]:
data.shape

(768, 9)

In [62]:
data.describe()

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age,label
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [63]:
data.isnull().values.any()

False

In [64]:
def plot_correlated(data, size=11):
    """
    Function to show plot of correlation between the data columns
    data : the data from pandas Dataframe
    size : the horizontal and vertical size of the plot
    
    Result : blue - cyan - yellow - red -darkred = less to more correlated columns = 0 ---- 1 
    """
    corr = data.corr()
    fig, ax = plt.subplots(figsize=(size, size))
    ax.matshow(corr)
    plt.xticks(range(len(corr.columns)), corr.columns)
    plt.yticks(range(len(corr.columns)), corr.columns)

In [65]:
data.corr()

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age,label
pregnant,1.000000,0.129459,0.141282,-0.081672,-0.073535,0.017683,-0.033523,0.544341,0.221898
glucose,0.129459,1.000000,0.152590,0.057328,0.331357,0.221071,0.137337,0.263514,0.466581
bp,0.141282,0.152590,1.000000,0.207371,0.088933,0.281805,0.041265,0.239528,0.065068
skin,-0.081672,0.057328,0.207371,1.000000,0.436783,0.392573,0.183928,-0.113970,0.074752
insulin,-0.073535,0.331357,0.088933,0.436783,1.000000,0.197859,0.185071,-0.042163,0.130548
bmi,0.017683,0.221071,0.281805,0.392573,0.197859,1.000000,0.140647,0.036242,0.292695
pedigree,-0.033523,0.137337,0.041265,0.183928,0.185071,0.140647,1.000000,0.033561,0.173844
age,0.544341,0.263514,0.239528,-0.113970,-0.042163,0.036242,0.033561,1.000000,0.238356
label,0.221898,0.466581,0.065068,0.074752,0.130548,0.292695,0.173844,0.238356,1.000000


In [66]:
feature_columns = ['pregnant', 'glucose', 'bp', 'skin', 'insulin', 'bmi', 'pedigree', 'age']
predicted_class = ['label']

X = data[feature_columns].values
y = data[predicted_class].values
#split_test_size =0.30

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = split_test_size, random_state=10)

In [67]:
label = data['label']
data.drop('label', axis=1, inplace=True)
X_p, y_p = data, label

In [68]:
# selection of relevant features
#label = data['Outcome']
#data.drop('', axis=1, inplace=True)

#X, y = data, label
#X_p = pd.read_csv("diabetes.csv", usecols = [0,1,2,3,4,5,6,7])
#y_p = pd.read_csv("diabetes.csv", usecols = ['Outcome'])


#X_p.head(5)



In [69]:
#X_train.shape

In [70]:
print(X)

[[  6.    148.     72.    ...  33.6     0.627  50.   ]
 [  1.     85.     66.    ...  26.6     0.351  31.   ]
 [  8.    183.     64.    ...  23.3     0.672  32.   ]
 ...
 [  5.    121.     72.    ...  26.2     0.245  30.   ]
 [  1.    126.     60.    ...  30.1     0.349  47.   ]
 [  1.     93.     70.    ...  30.4     0.315  23.   ]]


In [71]:
#from sklearn.model_selection import KFold 
kf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=10) 

for train_index, test_index in kf.split(X):
    #print("Train:", train_index, "Validation:",test_index)
    X_train, X_test = X[train_index], X[test_index] 
    y_train, y_test = y[train_index], y[test_index]

In [72]:
X_train.shape

(615, 8)

In [73]:
y_train.shape

(615, 1)

In [74]:
X_test.shape

(153, 8)

In [75]:
y_test.shape

(153, 1)

In [76]:
from sklearn.linear_model import LogisticRegression

log_reg_model = LogisticRegression(C=1.0, random_state=10)

log_reg_model.fit(X_train, y_train.ravel())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=10, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [77]:
lr_predict_train_data = log_reg_model.predict(X_train)

from sklearn import metrics

print("Accuracy = {0:.3f}".format(metrics.accuracy_score(y_train, lr_predict_train_data)))

Accuracy = 0.792


In [78]:
lr_predict_test_data = log_reg_model.predict(X_test)

from sklearn import metrics


print("Accuracy = {0:.3f}".format(metrics.accuracy_score(y_test, lr_predict_test_data)))

Accuracy = 0.752


In [79]:
print("Confusion matrix")

print("{0}".format(metrics.confusion_matrix(y_test, lr_predict_test_data, labels=[1,0])))

print("Classification report")

print("{0}".format(metrics.classification_report(y_test, lr_predict_test_data, labels=[1,0])))

Confusion matrix
[[34 30]
 [ 8 81]]
Classification report
             precision    recall  f1-score   support

          1       0.81      0.53      0.64        64
          0       0.73      0.91      0.81        89

avg / total       0.76      0.75      0.74       153



In [80]:
log_reg_model = LogisticRegression(C=1.0, class_weight="balanced", random_state=11)

log_reg_model.fit(X_train, y_train.ravel())

lr_predict_test_data = log_reg_model.predict(X_test)


In [81]:
print("Accuracy = {0:.3f}".format(metrics.accuracy_score(y_test, lr_predict_test_data)))

Accuracy = 0.791


In [82]:
print("Confusion matrix")

print("{0}".format(metrics.confusion_matrix(y_test, lr_predict_test_data, labels=[1,0])))

print("Classification report")

print("{0}".format(metrics.classification_report(y_test, lr_predict_test_data, labels=[1,0])))


Confusion matrix
[[54 10]
 [22 67]]
Classification report
             precision    recall  f1-score   support

          1       0.71      0.84      0.77        64
          0       0.87      0.75      0.81        89

avg / total       0.80      0.79      0.79       153



In [83]:
from sklearn.linear_model import LogisticRegressionCV
lrcv_model = LogisticRegressionCV(n_jobs=-1, random_state=10, Cs=10, cv=5, refit=True, class_weight="balanced")
lrcv_model.fit(X, y.ravel())

LogisticRegressionCV(Cs=10, class_weight='balanced', cv=5, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=-1, penalty='l2', random_state=10,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [84]:
lrcv_predict_test_data = lrcv_model.predict(X_test)

In [85]:
print("Accuracy = {0:.3f}".format(metrics.accuracy_score(y_test, lrcv_predict_test_data)))

Accuracy = 0.797


In [86]:
print("Confusion matrix")

print("{0}".format(metrics.confusion_matrix(y_test, lrcv_predict_test_data, labels=[1,0])))

print("Classification report")

print("{0}".format(metrics.classification_report(y_test, lrcv_predict_test_data, labels=[1,0])))


Confusion matrix
[[52 12]
 [19 70]]
Classification report
             precision    recall  f1-score   support

          1       0.73      0.81      0.77        64
          0       0.85      0.79      0.82        89

avg / total       0.80      0.80      0.80       153



In [87]:
from sklearn.ensemble  import GradientBoostingClassifier
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.preprocessing import binarize
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [88]:
%%time
# GridSearch procedure = cpu demanding
gradient_boost_eval = GradientBoostingClassifier(random_state=0)

params = {
    'learning_rate': [0.05, 0.1, 0.5],
    'max_features': [0.5, 1],
    'max_depth': [3, 4, 5] 
         }

grid_search = GridSearchCV(gradient_boost_eval, params, cv=10, scoring='roc_auc')
grid_search.fit(X_p,y_p)

print(grid_search.best_params_)
print(grid_search.best_score_)

{'learning_rate': 0.05, 'max_depth': 3, 'max_features': 0.5}
0.833971391263058
CPU times: user 1min 38s, sys: 52.5 ms, total: 1min 38s
Wall time: 1min 39s


In [89]:
# GBC belong to the Random Tree Classifier family
# a complex model who perform well on small Dataset
# We set our hyper-params with the best params providing by GridSearch
gradient_boost = GradientBoostingClassifier(learning_rate=0.05, max_depth=3, max_features=0.5, random_state=0)


In [90]:
#Cross Validation handle the .fit and .predict method
cross_val_score(gradient_boost, X_p, y_p, cv=10, scoring='roc_auc').mean()

0.833980056980057

In [91]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
#from sklearn.model_selection import KFold 
kf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=10) 

for train_index, test_index in kf.split(X_p):
    #print("Train:", train_index, "Validation:",test_index)
    X_train, X_test = X[train_index], X[test_index] 
    y_train, y_test = y[train_index], y[test_index]

In [92]:
#Adjusting the Threshold for improvement

In [93]:
gradient_boost_imp = GradientBoostingClassifier(learning_rate=0.05, max_depth=3, max_features=0.5, random_state=0)



gradient_boost_imp.fit(X_train, y_train)

/home/praveen/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=3,
              max_features=0.5, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=0, subsample=1.0, verbose=0,
              warm_start=False)

In [94]:
# Storing the predicted probabilities of classification
y_pred_prob = gradient_boost_imp.predict_proba(X_test)[:, 1]

In [95]:
# level of predictive precision
roc_auc_score(y_test, y_pred_prob)

0.8600852272727273

# using Keras

## Data Scaling 

We scale the data so as to increase the model accuracy.

In [96]:
scaler = MinMaxScaler()

scaled_x_train = scaler.fit_transform(X_train)
scaled_x_test = scaler.transform(X_test)


## KERAS DATA MODEL

We will use the following parameters in our implementation -

- Hidden Layers - 3 , each consisiting of 8 neurons.
- Activation - RELU for hidden, SOFTMAX for output layer.
- Optimizer - SGD
- Learning Rate Decay - 0.0001
- L2 Regularization

In [97]:
# Create Keras DNN Model

model = models.Sequential()

# Hyperparameters
hold_prob = 0.0
beta = 1e-6
alpha = 0.05
lr_decay = 0.01
iterations = 1000
validation_split = 0.5
opt_momentum = 0.8 # (Use only for SGD)

# Optimizer
opt = optimizers.SGD(lr=alpha, decay=lr_decay, momentum=opt_momentum, nesterov=True)

# First Layer
model.add(layers.Dense(input_dim=8, units=8, activation='relu'))

# Hidden Layers
model.add(layers.Dense(units=8, activation='relu', kernel_regularizer=keras.regularizers.l2(beta)))
model.add(layers.Dropout(hold_prob))

model.add(layers.Dense(units=8, activation='relu', kernel_regularizer=keras.regularizers.l2(beta)))
model.add(layers.Dropout(hold_prob))

model.add(layers.Dense(units=8, activation='relu', kernel_regularizer=keras.regularizers.l2(beta)))
model.add(layers.Dropout(hold_prob))

# Output Layer
model.add(layers.Dense(units=2, activation='softmax'))

# Compiling the Model
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(x=scaled_x_train, y=y_train, epochs=iterations, validation_split=validation_split)

Train on 346 samples, validate on 346 samples
Epoch 1/1000
346/346 [==============================] - 2s 5ms/step - loss: 0.6749 - acc: 0.5954 - val_loss: 0.6291 - val_acc: 0.6936
Epoch 2/1000
346/346 [==============================] - 0s 505us/step - loss: 0.6593 - acc: 0.6243 - val_loss: 0.6224 - val_acc: 0.6936
Epoch 3/1000
346/346 [==============================] - 0s 636us/step - loss: 0.6558 - acc: 0.6243 - val_loss: 0.6222 - val_acc: 0.6936
Epoch 4/1000
346/346 [==============================] - 0s 551us/step - loss: 0.6525 - acc: 0.6243 - val_loss: 0.6195 - val_acc: 0.6936
Epoch 5/1000
346/346 [==============================] - ETA: 0s - loss: 0.6395 - acc: 0.645 - 0s 555us/step - loss: 0.6477 - acc: 0.6243 - val_loss: 0.6168 - val_acc: 0.6936
Epoch 6/1000
346/346 [==============================] - 0s 528us/step - loss: 0.6409 - acc: 0.6503 - val_loss: 0.6056 - val_acc: 0.7023
Epoch 7/1000
346/346 [==============================] - 0s 463us/step - loss: 0.6334 - acc: 0.6272 - v

Epoch 60/1000
346/346 [==============================] - 0s 379us/step - loss: 0.4938 - acc: 0.7688 - val_loss: 0.4569 - val_acc: 0.7919
Epoch 61/1000
346/346 [==============================] - 0s 396us/step - loss: 0.4954 - acc: 0.7659 - val_loss: 0.4714 - val_acc: 0.8092
Epoch 62/1000
346/346 [==============================] - 0s 424us/step - loss: 0.4936 - acc: 0.7630 - val_loss: 0.4586 - val_acc: 0.8064
Epoch 63/1000
346/346 [==============================] - 0s 459us/step - loss: 0.4932 - acc: 0.7630 - val_loss: 0.4619 - val_acc: 0.8121
Epoch 64/1000
346/346 [==============================] - 0s 438us/step - loss: 0.4943 - acc: 0.7601 - val_loss: 0.4630 - val_acc: 0.8092
Epoch 65/1000
346/346 [==============================] - 0s 455us/step - loss: 0.4956 - acc: 0.7601 - val_loss: 0.4668 - val_acc: 0.8064
Epoch 66/1000
346/346 [==============================] - 0s 519us/step - loss: 0.4947 - acc: 0.7630 - val_loss: 0.4558 - val_acc: 0.7948
Epoch 67/1000
346/346 [==================

346/346 [==============================] - 0s 366us/step - loss: 0.4901 - acc: 0.7630 - val_loss: 0.4564 - val_acc: 0.8121
Epoch 120/1000
346/346 [==============================] - 0s 400us/step - loss: 0.4897 - acc: 0.7746 - val_loss: 0.4550 - val_acc: 0.8092
Epoch 121/1000
346/346 [==============================] - 0s 397us/step - loss: 0.4871 - acc: 0.7717 - val_loss: 0.4672 - val_acc: 0.8006
Epoch 122/1000
346/346 [==============================] - 0s 685us/step - loss: 0.4897 - acc: 0.7746 - val_loss: 0.4543 - val_acc: 0.8035
Epoch 123/1000
346/346 [==============================] - 0s 454us/step - loss: 0.4892 - acc: 0.7775 - val_loss: 0.4608 - val_acc: 0.8035
Epoch 124/1000
346/346 [==============================] - 0s 484us/step - loss: 0.4885 - acc: 0.7832 - val_loss: 0.4587 - val_acc: 0.8035
Epoch 125/1000
346/346 [==============================] - 0s 500us/step - loss: 0.4863 - acc: 0.7717 - val_loss: 0.4561 - val_acc: 0.8092
Epoch 126/1000
346/346 [=========================

346/346 [==============================] - 0s 350us/step - loss: 0.4853 - acc: 0.7717 - val_loss: 0.4559 - val_acc: 0.8035
Epoch 179/1000
346/346 [==============================] - 0s 345us/step - loss: 0.4819 - acc: 0.7775 - val_loss: 0.4567 - val_acc: 0.7977
Epoch 180/1000
346/346 [==============================] - 0s 500us/step - loss: 0.4842 - acc: 0.7688 - val_loss: 0.4531 - val_acc: 0.8035
Epoch 181/1000
346/346 [==============================] - 0s 369us/step - loss: 0.4823 - acc: 0.7803 - val_loss: 0.4572 - val_acc: 0.7977
Epoch 182/1000
346/346 [==============================] - 0s 530us/step - loss: 0.4817 - acc: 0.7890 - val_loss: 0.4521 - val_acc: 0.8006
Epoch 183/1000
346/346 [==============================] - 0s 479us/step - loss: 0.4834 - acc: 0.7717 - val_loss: 0.4560 - val_acc: 0.8006
Epoch 184/1000
346/346 [==============================] - 0s 498us/step - loss: 0.4877 - acc: 0.7746 - val_loss: 0.4647 - val_acc: 0.7948
Epoch 185/1000
346/346 [=========================

346/346 [==============================] - 0s 545us/step - loss: 0.4804 - acc: 0.7775 - val_loss: 0.4566 - val_acc: 0.8006
Epoch 238/1000
346/346 [==============================] - 0s 557us/step - loss: 0.4788 - acc: 0.7832 - val_loss: 0.4534 - val_acc: 0.8006
Epoch 239/1000
346/346 [==============================] - 0s 461us/step - loss: 0.4790 - acc: 0.7861 - val_loss: 0.4513 - val_acc: 0.7977
Epoch 240/1000
346/346 [==============================] - 0s 388us/step - loss: 0.4791 - acc: 0.7803 - val_loss: 0.4542 - val_acc: 0.8064
Epoch 241/1000
346/346 [==============================] - 0s 371us/step - loss: 0.4781 - acc: 0.7775 - val_loss: 0.4529 - val_acc: 0.8035
Epoch 242/1000
346/346 [==============================] - 0s 379us/step - loss: 0.4810 - acc: 0.7861 - val_loss: 0.4533 - val_acc: 0.8035
Epoch 243/1000
346/346 [==============================] - 0s 508us/step - loss: 0.4800 - acc: 0.7775 - val_loss: 0.4529 - val_acc: 0.8035
Epoch 244/1000
346/346 [=========================

...

346/346 [==============================] - 0s 317us/step - loss: 0.4641 - acc: 0.7919 - val_loss: 0.4566 - val_acc: 0.7890
Epoch 709/1000
346/346 [==============================] - 0s 344us/step - loss: 0.4635 - acc: 0.7977 - val_loss: 0.4554 - val_acc: 0.7890
Epoch 710/1000
346/346 [==============================] - 0s 363us/step - loss: 0.4636 - acc: 0.7919 - val_loss: 0.4549 - val_acc: 0.7890
Epoch 711/1000
346/346 [==============================] - 0s 326us/step - loss: 0.4638 - acc: 0.7890 - val_loss: 0.4561 - val_acc: 0.7890
Epoch 712/1000
346/346 [==============================] - 0s 351us/step - loss: 0.4655 - acc: 0.7861 - val_loss: 0.4545 - val_acc: 0.7919
Epoch 713/1000
346/346 [==============================] - 0s 339us/step - loss: 0.4632 - acc: 0.7948 - val_loss: 0.4556 - val_acc: 0.7890
Epoch 714/1000
346/346 [==============================] - ETA: 0s - loss: 0.4480 - acc: 0.816 - 0s 565us/step - loss: 0.4636 - acc: 0.7977 - val_loss: 0.4547 - val_acc: 0.7890
Epoch 715/1

346/346 [==============================] - 0s 673us/step - loss: 0.4621 - acc: 0.7948 - val_loss: 0.4547 - val_acc: 0.7890
Epoch 768/1000
346/346 [==============================] - 0s 627us/step - loss: 0.4624 - acc: 0.7919 - val_loss: 0.4552 - val_acc: 0.7890
Epoch 769/1000
346/346 [==============================] - 0s 503us/step - loss: 0.4624 - acc: 0.7948 - val_loss: 0.4559 - val_acc: 0.7919
Epoch 770/1000
346/346 [==============================] - 0s 426us/step - loss: 0.4625 - acc: 0.7948 - val_loss: 0.4547 - val_acc: 0.7890
Epoch 771/1000
346/346 [==============================] - 0s 407us/step - loss: 0.4622 - acc: 0.7919 - val_loss: 0.4554 - val_acc: 0.7919
Epoch 772/1000
346/346 [==============================] - 0s 479us/step - loss: 0.4626 - acc: 0.7919 - val_loss: 0.4557 - val_acc: 0.7919
Epoch 773/1000
346/346 [==============================] - 0s 707us/step - loss: 0.4615 - acc: 0.7919 - val_loss: 0.4543 - val_acc: 0.7890
Epoch 774/1000
346/346 [=========================

346/346 [==============================] - 0s 610us/step - loss: 0.4610 - acc: 0.7948 - val_loss: 0.4564 - val_acc: 0.7919
Epoch 827/1000
346/346 [==============================] - 0s 663us/step - loss: 0.4608 - acc: 0.7948 - val_loss: 0.4562 - val_acc: 0.7919
Epoch 828/1000
346/346 [==============================] - 0s 539us/step - loss: 0.4609 - acc: 0.7948 - val_loss: 0.4565 - val_acc: 0.7919
Epoch 829/1000
346/346 [==============================] - 0s 425us/step - loss: 0.4611 - acc: 0.7948 - val_loss: 0.4558 - val_acc: 0.7919
Epoch 830/1000
346/346 [==============================] - 0s 602us/step - loss: 0.4608 - acc: 0.7948 - val_loss: 0.4560 - val_acc: 0.7890
Epoch 831/1000
346/346 [==============================] - 0s 426us/step - loss: 0.4603 - acc: 0.7948 - val_loss: 0.4570 - val_acc: 0.7919
Epoch 832/1000
346/346 [==============================] - 0s 459us/step - loss: 0.4617 - acc: 0.7919 - val_loss: 0.4566 - val_acc: 0.7919
Epoch 833/1000
346/346 [=========================

346/346 [==============================] - 0s 332us/step - loss: 0.4605 - acc: 0.7919 - val_loss: 0.4558 - val_acc: 0.7890
Epoch 886/1000
346/346 [==============================] - 0s 326us/step - loss: 0.4598 - acc: 0.7948 - val_loss: 0.4570 - val_acc: 0.7890
Epoch 887/1000
346/346 [==============================] - 0s 341us/step - loss: 0.4615 - acc: 0.7832 - val_loss: 0.4570 - val_acc: 0.7890
Epoch 888/1000
346/346 [==============================] - 0s 325us/step - loss: 0.4598 - acc: 0.7919 - val_loss: 0.4549 - val_acc: 0.7890
Epoch 889/1000
346/346 [==============================] - 0s 315us/step - loss: 0.4598 - acc: 0.7919 - val_loss: 0.4561 - val_acc: 0.7890
Epoch 890/1000
346/346 [==============================] - 0s 325us/step - loss: 0.4610 - acc: 0.7919 - val_loss: 0.4564 - val_acc: 0.7890
Epoch 891/1000
346/346 [==============================] - 0s 326us/step - loss: 0.4598 - acc: 0.7948 - val_loss: 0.4558 - val_acc: 0.7919
Epoch 892/1000
346/346 [=========================

346/346 [==============================] - 0s 318us/step - loss: 0.4584 - acc: 0.7948 - val_loss: 0.4573 - val_acc: 0.7890
Epoch 945/1000
346/346 [==============================] - 0s 311us/step - loss: 0.4588 - acc: 0.7890 - val_loss: 0.4577 - val_acc: 0.7890
Epoch 946/1000
346/346 [==============================] - 0s 372us/step - loss: 0.4588 - acc: 0.7948 - val_loss: 0.4572 - val_acc: 0.7861
Epoch 947/1000
346/346 [==============================] - 0s 282us/step - loss: 0.4589 - acc: 0.7919 - val_loss: 0.4571 - val_acc: 0.7861
Epoch 948/1000
346/346 [==============================] - 0s 314us/step - loss: 0.4588 - acc: 0.7948 - val_loss: 0.4576 - val_acc: 0.7890
Epoch 949/1000
346/346 [==============================] - 0s 324us/step - loss: 0.4589 - acc: 0.7948 - val_loss: 0.4564 - val_acc: 0.7948
Epoch 950/1000
346/346 [==============================] - 0s 302us/step - loss: 0.4583 - acc: 0.7948 - val_loss: 0.4572 - val_acc: 0.7890
Epoch 951/1000
346/346 [=========================

In [104]:
predictions = model.predict_classes(scaled_x_test)
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       0.72      0.82      0.77        44
          1       0.69      0.56      0.62        32

avg / total       0.71      0.71      0.70        76



## Accuracy on test data set


In [105]:
roc_auc_score(y_test,predictions )

0.6903409090909091

## Accuracy on train data set

In [106]:
predictions = model.predict_classes(scaled_x_train)

In [107]:
roc_auc_score(y_train,predictions)

0.745948557835266